# Domineering

In [1]:
%pip install numpy
import numpy as np
from IPython.core.display import display, HTML

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\1_Workspaces\1_Phd_Projects\GKI_23\venv\Scripts\python.exe -m pip install --upgrade pip' command.
C:\Users\Max\AppData\Local\Temp\ipykernel_35556\4185587730.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Modellierung der Zustände und des Spielgraphen

In [2]:
def create_regular_board(width: int, height: int) -> np.array:
    "Erstellt ein rechteckiges Spielfeld mit der angegebenen Breite und Höhe."
    return np.zeros((height, width), dtype=np.int8)


def create_irregular_board(widths: list[int]) -> np.array:
    "Erstellt ein irreguläres Spielfeld von Zeilen unterschiedlicher Breite."
    board = create_regular_board(np.max(widths), len(widths))
    for y, w in enumerate(widths):
        board[y, w:] = 4
    return board


def create_board(w, h=None, return_id: bool = False) -> np.array:
    "Kombination von create_regular_board und create_irregular_board."
    if isinstance(w, int):
        if h is None:
            h = w
        id = f"{w}x{h}"
        b = create_regular_board(w, h)
    else:
        id = "+".join([str(x) for x in w])
        b = create_irregular_board(w)

    if return_id:
        return b, id
    return b


def successors(board: np.array, player: int) -> list[np.array]:
    """Gibt eine Liste mögliche Folgezustände für ein gegebenes Spielfeld und einen aktiven Spieler zurück.
       player = +1: H ist am Zug
       player = -1: V ist am Zug"""
    height, width = board.shape
    boards = []

    # Spieler +1 (H) ist am zug:
    if player == 1:
        for y in range(height):
            for x in range(width - 1):
                # wenn die Zelle leer ist und die rechte Nachbarzelle auch leer ist:
                if board[y, x] == 0 and board[y, x + 1] == 0:
                    succ = np.copy(board)
                    succ[y, x:x + 2] = 1  # für +1 (H) werden die Zellen des Feldes auf +1 gesetzt
                    boards.append(succ)
    # Spieler -1 (V) ist am zug:
    elif player == -1:
        for x in range(width):
            for y in range(height - 1):
                # wenn die Zelle leer ist und die untere Nachbarzelle auch leer ist:
                if board[y, x] == 0 and board[y + 1, x] == 0:
                    succ = np.copy(board)
                    succ[y:y + 2, x] = -1  # für +1 (V) werden die Zellen des Feldes auf -1 gesetzt
                    boards.append(succ)
    else:
        raise Exception(f"Invalid player {player}.")
    return boards


def flip_board(board: np.array) -> np.array:
  return np.transpose(-board)

In [3]:
def board_html(board: np.array) -> str:
  height, width = board.shape
  
  out = "<table style='margin:2px;'>"
  
  for y in range(height):
    out += "<tr>"
    for x in range(width):
      b = board[y, x]
      if abs(b) == 4:
        out += "<td style='width:1em;height:1em;margin:0;padding:0;'>"
      else:
        if b == 1: 
          c = "red"
        elif b == -1:
          c = "blue"
        else:
          c = "white"
        out += "<td style='width:1em;height:1em;background-color:" + c + ";margin:0;padding:0;border:1px solid black !important;'>"
      out += "</td>"
    out += "</tr>"
  
  out += "</table>"
  return out

def draw_board(board: np.array):
  display(HTML(board_html(board)))
  
def draw_boards(boards: list[np.array]):
  out = "<div style='position:relative;display:flex;overflow:auto;flex-wrap:wrap;'>"
  for board in boards:
    out += board_html(board)
  out += "</div>"
  display(HTML(out))

## Visualisierung der Startzüge

In [4]:
b, id = create_board(3, return_id=True)
print(f"Startzüge für H auf {id}:")
draw_boards(successors(b, 1))
print(f"Startzüge für V auf {id}:")
draw_boards(successors(b, -1))

b, id = create_board([4,1,2,3], return_id=True)
print(f"Startzüge für H auf {id}:")
draw_boards(successors(b, 1))
print(f"Startzüge für V auf {id}:")
draw_boards(successors(b, -1))

Startzüge für H auf 3x3:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Startzüge für V auf 3x3:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Startzüge für H auf 4+1+2+3:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


Startzüge für V auf 4+1+2+3:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


## Implementation von Minimax- und $\alpha$-$\beta$-Suche

In [5]:
# Die min- und max-search Routinen aus der VL sind hier in einer Funktion kombiniert:
def min_max_search(
        s,
        player: int,
        a: int = None, b: int = None,
        with_pruning: bool = False,
        return_leaf_count: bool = False,
        return_moves: bool = False
):
    """Führt eine Min-Max-Suche (oder Alpha-Beta-Suche) auf dem gegebenen Spielfeld s durch.

    s ist ein Spielfeld (aktueller Zustand).
    player ist der aktive Spieler:
        player = +1: H ist am Zug
        player = -1: V ist am Zug
    a und b sind die Grenzen für die Alpha-Beta-Suche.

    with_pruning = True: Alpha-Beta-Suche wird verwendet.
    return_leaf_count = True: Anzahl der Blätter wird zurückgegeben.
    return_moves = True: Liste der Züge wird zurückgegeben.
    """

    # Hilfsfunktion, die das Ergebnis der Suche erzeugt:
    def make_result(v: int, leaf_count: int, moves):
        "Erzeugt das Ergebnis der Suche."
        if not return_leaf_count and not return_moves:
            return v
        res = (v,)
        if return_leaf_count:
            res += (leaf_count,)
        if return_moves:
            res += (moves,)
        return res

    # Falls a oder b nicht angegeben sind, werden sie mit -∞ bzw. +∞ initialisiert:
    if a is None:
        a = np.NINF * player
    if b is None:
        b = np.PINF * player

    # Die moves Liste enthält die Züge, die zum aktuellen Zustand geführt haben:
    # (initial: nur der aktuelle Zustand) falls return_moves = True
    moves = [s]

    # Liste der möglichen Folgezustände werden erzeugt:
    succs = successors(s, player)

    # Aktueller Spieler kann nicht mehr legen wir sind an einem Endzustand:
    if len(succs) == 0:
        # Falls +1 (H) am Zug ist: Ergebnis -1 (V gewinnt)
        # Falls -1 (V) am Zug ist: Ergebnis +1 (H gewinnt)
        v = -1 * player
        return make_result(v=v, leaf_count=1, moves=moves)

    # +1 (H) möchte Ergebnis +1 (H gewinnt) => Maximales Ergebnis
    # -1 (V) möchte Ergebnis -1 (V gewinnt) => Minimales Ergebnis
    if player == 1:
        select_best = max
    else:
        select_best = min

    # H ist am Zug: best_v wird mit -∞ initialisiert sodass best_v < v für jeden zukünftigen Zustand gilt
    # V ist am Zug: best_v wird mit +∞ initialisiert sodass best_v < v für jeden zukünftigen Zustand gilt
    best_v = np.NINF * player

    # Hilfsvariablen:
    best_moves_candidate = []  # Liste der besten Züge (mit denen best_v erreicht wird)
    lf_count = 0  # Anzahl der Blätter für Endresultat:

    # Eigentliche Suche. Für jeden Folgezustand wird die Suche rekursiv aufgerufen:
    for succ in succs:
        # Rekursiver Aufruf: Suche für den Folgezustand mit dem anderen Spieler und aktuellem Alpha/Beta:
        rs = min_max_search(
            succ, -player, b, a,
            with_pruning=with_pruning,
            return_leaf_count=return_leaf_count,
            return_moves=return_moves)

        if return_leaf_count or return_moves:
            v = rs[0]
            rs = rs[1:]
            if return_leaf_count:
                lf_count += rs[0]
                rs = rs[1:]
            if return_moves:
                moves_candidate = rs[0]
        else:
            v = rs

        # Falls der Folgezustand besser ist als der bisher beste Zustand:
        if best_v != select_best(v, best_v):
            best_v = v
            if return_moves:
                best_moves_candidate = moves_candidate

        # Falls Alpha-Beta-Suche verwendet wird:
        if with_pruning:
            if v == b or select_best(v, b) != b:
                return make_result(v, lf_count, moves + moves_candidate)
            a = select_best(best_v, a)

    return make_result(best_v, lf_count, moves + best_moves_candidate)

## Kategorisierung der Spieltypen

In [6]:
def print_result(player, best_v, lf_count, best_moves):
  starter = "H" if player == 1 else "V"
  winner = "H" if best_v == 1 else "V"

  print(f"Startspieler: {starter}, Gewinner: {winner}, Betrachtete Spielverläufe: {lf_count}")
  print("(Eine) optimale Zugsequenz für beide Spieler:")
  draw_boards(best_moves)
  return best_v

def eval_player(board: np.array, player: int):
  return (player, *min_max_search(
    board, player, 
    with_pruning=True,
    return_leaf_count=True, return_moves=True))

def eval_board(w, h=None, square_opt=True):
  board, id = create_board(w, h, return_id=True)
  is_square = square_opt and isinstance(w, int) and (h is None or w == h)
  print(f"Evaluation des {id} Spielfelds:")
  result_H = eval_player(board, +1)
  _, winner_H, lf_count_H, moves_H = result_H
  print_result(*result_H)
  if is_square:
    print("Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.")
    winner_V = print_result(
      -1, -winner_H, lf_count_H,
      [flip_board(b) for b in moves_H])
  else:
    winner_V = print_result(*eval_player(board, -1))
  if winner_H == 1 and winner_V == 1:
    print("Spieler H gewinnt immer.")
  elif winner_H == -1 and winner_V == -1:
    print("Spieler V gewinnt immer.")
  elif winner_H == 1 and winner_V == -1:
    print("Der Spieler, der den ersten Zug macht, gewinnt.")
  else:
    print("Der nachziehende Spieler gewinnt.")

In [16]:
eval_board(1)

Evaluation des 1x1 Spielfelds:
Startspieler: H, Gewinner: V, Betrachtete Spielverläufe: 1
(Eine) optimale Zugsequenz für beide Spieler:


""


Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.
Startspieler: V, Gewinner: H, Betrachtete Spielverläufe: 1
(Eine) optimale Zugsequenz für beide Spieler:


""


Der nachziehende Spieler gewinnt.


In [8]:
eval_board([1,1])

Evaluation des 1+1 Spielfelds:
Startspieler: H, Gewinner: V, Betrachtete Spielverläufe: 1
(Eine) optimale Zugsequenz für beide Spieler:


""
""


Startspieler: V, Gewinner: V, Betrachtete Spielverläufe: 1
(Eine) optimale Zugsequenz für beide Spieler:


""
""
""
""


Spieler V gewinnt immer.


In [9]:
eval_board(2)

Evaluation des 2x2 Spielfelds:
Startspieler: H, Gewinner: H, Betrachtete Spielverläufe: 2
(Eine) optimale Zugsequenz für beide Spieler:


,
,
,
,


Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.
Startspieler: V, Gewinner: V, Betrachtete Spielverläufe: 2
(Eine) optimale Zugsequenz für beide Spieler:


,
,
,
,


Der Spieler, der den ersten Zug macht, gewinnt.


In [10]:
eval_board(3)

Evaluation des 3x3 Spielfelds:
Startspieler: H, Gewinner: H, Betrachtete Spielverläufe: 21
(Eine) optimale Zugsequenz für beide Spieler:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.
Startspieler: V, Gewinner: V, Betrachtete Spielverläufe: 21
(Eine) optimale Zugsequenz für beide Spieler:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Der Spieler, der den ersten Zug macht, gewinnt.


In [11]:
eval_board(4)

Evaluation des 4x4 Spielfelds:
Startspieler: H, Gewinner: H, Betrachtete Spielverläufe: 787
(Eine) optimale Zugsequenz für beide Spieler:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.
Startspieler: V, Gewinner: V, Betrachtete Spielverläufe: 787
(Eine) optimale Zugsequenz für beide Spieler:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


Der Spieler, der den ersten Zug macht, gewinnt.


In [12]:
eval_board(3,4)

Evaluation des 3x4 Spielfelds:
Startspieler: H, Gewinner: V, Betrachtete Spielverläufe: 124
(Eine) optimale Zugsequenz für beide Spieler:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Startspieler: V, Gewinner: V, Betrachtete Spielverläufe: 96
(Eine) optimale Zugsequenz für beide Spieler:


,,
,,
,,
,,
,,
,,
,,
,,
,,
,,
,,


Spieler V gewinnt immer.


In [13]:
eval_board(2,5)

Evaluation des 2x5 Spielfelds:
Startspieler: H, Gewinner: H, Betrachtete Spielverläufe: 28
(Eine) optimale Zugsequenz für beide Spieler:


,
,
,
,
,
,
,
,
,
,
,


Startspieler: V, Gewinner: H, Betrachtete Spielverläufe: 42
(Eine) optimale Zugsequenz für beide Spieler:


,
,
,
,
,
,
,
,
,
,
,


Spieler H gewinnt immer.


In [14]:
eval_board([4,1,2,3])

Evaluation des 4+1+2+3 Spielfelds:
Startspieler: H, Gewinner: H, Betrachtete Spielverläufe: 29
(Eine) optimale Zugsequenz für beide Spieler:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


Startspieler: V, Gewinner: H, Betrachtete Spielverläufe: 12
(Eine) optimale Zugsequenz für beide Spieler:


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


Spieler H gewinnt immer.


In [15]:
eval_board(5)

Evaluation des 5x5 Spielfelds:
Startspieler: H, Gewinner: V, Betrachtete Spielverläufe: 1123527
(Eine) optimale Zugsequenz für beide Spieler:


,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


Quadratisches Spielfeld => Auswertung für Spieler V kann aus H gefolgert werden.
Startspieler: V, Gewinner: H, Betrachtete Spielverläufe: 1123527
(Eine) optimale Zugsequenz für beide Spieler:


,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


Der nachziehende Spieler gewinnt.
